In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

retail_data = pd.read_csv('/content/OnlineRetail.csv', encoding= 'unicode_escape')
retail_data['InvoiceDate'] = pd.to_datetime(retail_data['InvoiceDate'], errors = 'coerce')
uk_data = retail_data.query("Country=='United Kingdom'").reset_index(drop=True)
uk_data.shape

(495478, 8)

In [ ]:
from datetime import datetime, timedelta,date
t1 = pd.Timestamp("2011-06-01 00:00:00.054000")
t2 = pd.Timestamp("2011-03-01 00:00:00.054000")
t3 = pd.Timestamp("2011-12-01 00:00:00.054000")
uk_data_3m = uk_data[(uk_data.InvoiceDate < t1) & (uk_data.InvoiceDate >= t2)].reset_index(drop=True)
uk_data_6m = uk_data[(uk_data.InvoiceDate >= t1) & (uk_data.InvoiceDate < t3)].reset_index(drop=True)


In [ ]:
uk_data_3m['revenue'] = uk_data_3m['UnitPrice'] * uk_data_3m['Quantity']
max_date = uk_data_3m['InvoiceDate'].max() + timedelta(days=1)
rfm_data = uk_data_3m.groupby(['CustomerID']).agg({
        'InvoiceDate': lambda x: (max_date - x.max()).days,
        'InvoiceNo': 'count',
        'revenue': 'sum'})

In [ ]:
rfm_data.rename(columns={'InvoiceDate': 'Recency',
                         'InvoiceNo': 'Frequency',
                         'revenue': 'MonetaryValue'}, inplace=True)

In [ ]:
rfm_data

,Recency,Frequency,MonetaryValue
CustomerID,,,
12747.0,7,35,1082.09
12748.0,1,582,4336.73
12749.0,8,54,782.10
12821.0,23,6,92.72
12823.0,63,1,459.00
...,...,...,...
18272.0,21,59,966.74
18273.0,66,1,51.00
18280.0,86,10,180.60


In [ ]:
r_grp = pd.qcut(rfm_data['Recency'], q=4, labels=range(3,-1,-1))
f_grp = pd.qcut(rfm_data['Frequency'], q=4, labels=range(0,4))
m_grp = pd.qcut(rfm_data['MonetaryValue'], q=4, labels=range(0,4))
rfm_data = rfm_data.assign(R=r_grp.values).assign(F=f_grp.values).assign(M=m_grp.values)
rfm_data['R'] = rfm_data['R'].astype(int)
rfm_data['F'] = rfm_data['F'].astype(int)
rfm_data['M'] = rfm_data['M'].astype(int)
rfm_data['RFMScore'] = rfm_data['R'] + rfm_data['F'] + rfm_data['M']
rfm_data.groupby('RFMScore')['Recency','Frequency','MonetaryValue'].mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  if __name__ == '__main__':


,Recency,Frequency,MonetaryValue
RFMScore,,,
0,76.633588,5.206107,94.023359
1,58.500000,9.074074,158.990864
2,54.380208,13.968750,239.346354
3,38.091633,16.605578,271.437331
4,36.647619,24.180952,363.867000
5,34.000000,33.558252,535.423204
6,28.516432,44.920188,742.487324
7,18.431138,51.317365,974.310120
8,14.316547,80.064748,2192.578568


In [ ]:
rfm_data['Segment'] = 'Low-Value'
rfm_data.loc[rfm_data['RFMScore']>4,'Segment'] = 'Mid-Value' 
rfm_data.loc[rfm_data['RFMScore']>6,'Segment'] = 'High-Value' 
rfm_data = rfm_data.reset_index()

In [ ]:
uk_data_6m['revenue'] = uk_data_6m['UnitPrice'] * uk_data_6m['Quantity']
revenue_6m = uk_data_6m.groupby(['CustomerID']).agg({
        'revenue': 'sum'})
revenue_6m.rename(columns={'revenue': 'Revenue_6m'}, inplace=True)

In [ ]:
revenue_6m = revenue_6m.reset_index()

In [ ]:
merged_data = pd.merge(rfm_data, revenue_6m, how="left")
merged_data.fillna(0)

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFMScore,Segment,Revenue_6m
0,12747.0,7,35,1082.09,3,2,3,8,High-Value,1666.11
1,12748.0,1,582,4336.73,3,3,3,9,High-Value,18679.01
2,12749.0,8,54,782.10,3,3,3,9,High-Value,2323.04
3,12821.0,23,6,92.72,2,0,0,2,Low-Value,0.00
4,12823.0,63,1,459.00,0,0,2,2,Low-Value,765.00
...,...,...,...,...,...,...,...,...,...,...
1835,18272.0,21,59,966.74,2,3,3,8,High-Value,1730.16
1836,18273.0,66,1,51.00,0,0,0,0,Low-Value,102.00
1837,18280.0,86,10,180.60,0,0,0,0,Low-Value,0.00
1838,18283.0,9,100,217.15,3,3,1,7,High-Value,1351.83


In [ ]:
merged_data = merged_data[merged_data['Revenue_6m']<merged_data['Revenue_6m'].quantile(0.99)]

from sklearn.cluster import KMeans
#creating 3 clusters
kmeans = KMeans(n_clusters=3)
kmeans.fit(merged_data[['Revenue_6m']])
merged_data['LTVCluster'] = kmeans.predict(merged_data[['Revenue_6m']])

In [ ]:
merged_data.groupby('LTVCluster')['Revenue_6m'].describe()

,count,mean,std,min,25%,50%,75%,max
LTVCluster,,,,,,,,
0,186.0,4137.019892,1477.396388,2503.30,2964.2275,3635.200,4987.535,8910.04
1,1170.0,828.670189,621.405282,-609.40,324.4675,663.525,1245.465,2447.57
2,21.0,14123.309048,3653.311262,9313.18,11464.6700,12913.990,16756.310,20530.00


In [ ]:
feature_data = pd.get_dummies(merged_data)
feature_data.reset_index()
feature_data.head(5)

,CustomerID,Recency,Frequency,MonetaryValue,R,F,M,RFMScore,Revenue_6m,LTVCluster,Segment_High-Value,Segment_Low-Value,Segment_Mid-Value
0,12747.0,7,35,1082.09,3,2,3,8,1666.11,1,1,0,0
1,12748.0,1,582,4336.73,3,3,3,9,18679.01,2,1,0,0
2,12749.0,8,54,782.10,3,3,3,9,2323.04,1,1,0,0
4,12823.0,63,1,459.00,0,0,2,2,765.00,1,0,1,0
7,12836.0,28,62,814.71,1,3,3,7,951.46,1,1,0,0


In [ ]:

from sklearn.metrics import classification_report,confusion_matrix
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

# feature_data = pd.get_dummies(merged_data)
X = feature_data.drop(['LTVCluster', 'Revenue_6m'], axis=1)
y = feature_data['LTVCluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
xgb_classifier = xgb.XGBClassifier(max_depth=5, objective='multi:softprob')
xgb_model = xgb_classifier.fit(X_train, y_train)
acc = xgb_model.score(X_test,y_test)
print(acc)

0.9057971014492754


In [ ]:
y_pred = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95       121
           1       1.00      0.50      0.67         2
           2       0.67      0.27      0.38        15

    accuracy                           0.91       138
   macro avg       0.86      0.59      0.67       138
weighted avg       0.89      0.91      0.89       138

